In [9]:
import re
from zxcvbn import zxcvbn
import sqlite3
import logging

# Setup logging for automation alerts
logging.basicConfig(level=logging.INFO)

def check_length(password: str) -> bool:
    """Checks if password meets minimum length of 12 characters."""
    return len(password) >= 12

def check_diversity(password: str) -> dict:
    """Uses regex to verify character types: uppercase, lowercase, digits, symbols."""
    patterns = {
        'uppercase': re.compile(r'[A-Z]'),
        'lowercase': re.compile(r'[a-z]'),
        'digits': re.compile(r'\d'),
        'symbols': re.compile(r'[!@#$%^&*(),.?":{}|<>]')
    }
    return {k: bool(p.search(password)) for k, p in patterns.items()}

def check_common_words(password: str) -> bool:
    """Queries SQLite DB for matches against common weak passwords."""
    try:
        conn = sqlite3.connect('weak_passwords.db')
        cursor = conn.cursor()
        cursor.execute("SELECT 1 FROM weak_passwords WHERE password = ?", (password,))
        result = cursor.fetchone()
        conn.close()
        return result is not None  # True if weak
    except sqlite3.Error as e:
        logging.error(f"Database error: {e}")
        return False

def score_password(password: str) -> tuple:
    """Combines zxcvbn guesses_log10 with custom checks for overall score (0-4: weak to strong)."""
    try:
        results = zxcvbn(password)
        # Use guesses_log10 instead of entropy
        guesses_log10 = results.get('guesses_log10', 0)  # Fallback to 0 if key missing
        diversity = check_diversity(password)
        num_types = sum(diversity.values())
        custom_score = 0
        if check_length(password):
            custom_score += 1
        if num_types >= 3:
            custom_score += 1
        if not check_common_words(password):
            custom_score += 1
        if not re.search(r'(.)\1{2,}', password):  # No repeating chars
            custom_score += 1
        if not re.search(r'(\d\d|abc|qwerty)', password):  # No sequences
            custom_score += 1
        # Normalize score using guesses_log10 (e.g., >10 is strong)
        overall_score = min(4, (guesses_log10 / 10) + custom_score)
        strength = ['Weak', 'Medium', 'Strong', 'Very Strong'][int(overall_score)]
        suggestions = results.get('feedback', {}).get('suggestions', [])
        return overall_score, strength, suggestions
    except Exception as e:
        logging.error(f"Error in score_password: {e}")
        return 0, "Weak", ["Unable to analyze password; try again."]

def suggest_improvements(password: str) -> list:
    """Generates tips based on failed checks."""
    tips = []
    if not check_length(password):
        tips.append("Increase length to at least 12 characters.")
    diversity = check_diversity(password)
    missing = [k for k, v in diversity.items() if not v]
    if missing:
        tips.append(f"Add {', '.join(missing)} characters.")
    if check_common_words(password):
        tips.append("Avoid common words; use unique phrases.")
    return tips

# Example usage with logging
if __name__ == "__main__":
    pwd = input("Enter password: ")
    score, strength, zxcvbn_tips = score_password(pwd)
    tips = suggest_improvements(pwd)
    print(f"Score: {score:.1f}/4 - {strength}")
    print("Suggestions:", tips + zxcvbn_tips)
    if score < 2:
        logging.warning(f"Weak password detected: {pwd[:3]}***")

ERROR:root:Database error: no such table: weak_passwords
ERROR:root:Error in score_password: list index out of range
ERROR:root:Database error: no such table: weak_passwords


Score: 0.0/4 - Weak
Suggestions: ['Unable to analyze password; try again.']
